In [2]:
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import os
import pandas as pd
import requests
import json
import time
import datetime
#spotify API works with 64base urls
import base64

In [36]:
df = pd.read_csv("../Data/01 billboard_scraped.csv", index_col=0)
df

,artist,location_at_chart,year_of_billboard
song_name,,,
heat waves,glass animals,1,2022
as it was,harry styles,2,2022
stay,the kid laroi & justin bieber,3,2022
easy on me,adele,4,2022
shivers,ed sheeran,5,2022
...,...,...,...
brave,sara bareilles,96,2013
let her go,passenger,97,2013
runnin' outta moonlight,randy houser,98,2013


# spotify requests function
setting the relevant data that needed to get an access token for spotify API requests

In [4]:
def spotify_req():
    client_id = "0cd5c209ad8d46b291518e5f5243ba22"
    client_secret = "91b4c2782052423493edadd09067d4a1"
    client_creds=f"{client_id}:{client_secret}"
    client_creds_b64=base64.b64encode(client_creds.encode())
    token_url="https://accounts.spotify.com/api/token"
    method = "POST"
    token_data = {
        "grant_type":"client_credentials"   
    }
    token_headers={
        "Authorization":f"Basic {client_creds_b64.decode()}"# <base64 encoded client_id:client_secret>
    }
    r=requests.post(token_url,data=token_data,headers=token_headers)
    print(r.json())
    valid_request = r.status_code in range(200,299) #return's boolean 
    # Set the authorization header with the access token
    headers = {
        "Authorization": f"Bearer {r.json()['access_token']}"
    }
    return headers
headers = spotify_req()

{'access_token': 'BQCEIhyYdookTl1ucaKcRDNK-slDIa7rXVDxNenopX31JxnMRtwX5XID0e9OBzjKX2OnORkkULntlgkRMVIa5dujBo8Vmy3whbg5KBAA6LmyTM55Kjm6', 'token_type': 'Bearer', 'expires_in': 3600}


# Billboard songs to thier Spotify ID's 
aquiring spotify's ID for each song from the billboard charts by making an API request passng the song names query as parameter

In [34]:
# Set up the API base URL and the API endpoint for searching for songs
api_base_url = "https://api.spotify.com"
search_endpoint = "/v1/search"

In [64]:
# setting a loop for extracting spotify track id for ecah song_name in data

track_ids = []
for item in df["song_name"]:
    query = item

    # Set up the search parameters
    params = {
        "q": query,
        "type": "track"
    }
    
    # Send the request to the API
    response = requests.get(api_base_url + search_endpoint, headers=headers, params=params)

    # Check the status code to make sure the request was successful
    if response.status_code not in range(200,299):
        print(response.headers)
        break
    else:
        # add the data returned by the API to ids array
        data = json.loads(response.text)
        track_ids.append(data["tracks"]["items"][0]["id"])
    time.sleep(1)
df["spotify_track_id"] = track_ids

In [68]:
df.tail()

,song_name,artist,location_at_chart,year_of_billboard,spotify_track_id
194,are you gonna kiss me or not,thompson square,96,2011,2TAcwD5KcV9uY3O1tLGV15
195,animal,neon trees,97,2011,1fBl642IhJOE5U319Gy2Go
196,you and tequila,kenny chesney featuring grace potter,98,2011,1t2hJFgJyUPD2crOafMUEk
197,colder weather,zac brown band,99,2011,3U8dHeggJ8IBe0UCb1gbyB
198,my last,big sean featuring chris brown,100,2011,70tFuqBcduJv15bEnOPRTh


# add audio features 
setting the endpoint to be the get tracks audio features endpoint, making lists for every feature and cheaking for missing data that the API request did not return 

In [69]:
# set the audio features end point
audio_features_endpoint = "/v1/audio-features"

#set the arrays for the audio features data response
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms = []
time_signature = []

#set loop for extracting the audio featureas data for each track in df
for item in ids_split_list:
    comma_separated_string = ','.join(item)
    response = requests.get(api_base_url + audio_features_endpoint + '/?ids=' + f'{comma_separated_string}' , headers=headers)
#     print(json.loads(response.text))
    data = json.loads(response.text)
    if response.status_code in range(200,299):
        print(response.headers)
        print(response.headers.get('Retry-After'))
        break
        
    else:
        for song in data['audio_features']:
            danceability.append(song["danceability"])
            energy.append(song["energy"])
            key.append(song["key"])
            loudness.append(song["loudness"])
            mode.append(song["mode"])
            speechiness.append(song["speechiness"])
            acousticness.append(song["acousticness"])
            instrumentalness.append(song["instrumentalness"])
            liveness.append(song["liveness"])
            valence.append(song["valence"])
            tempo.append(song["tempo"])
            duration_ms.append(song["duration_ms"])
            time_signature.append(song["time_signature"])
    time.sleep(4.5)

In [ ]:
#append new columns to df
new_df = df.assign(**{"danceability": danceability,"energy": energy, "key": key, "loudness": loudness,
             "mode": mode, "speechiness": speechiness, "acousticness": acousticness, "instrumentalness":instrumentalness,
            "liveness": liveness, "valence": valence, "tempo": tempo, "duration_ms": duration_ms,
             "time_signature": time_signature})

In [59]:
new_d.tail()
# new_d.to_csv("../Data/02 billboard_scraped_w featuers.csv")

,song_name,artist,location_at_chart,year_of_billboard,spotify_track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,entered_biilboard
994,brave,sara bareilles,96,2013,6Uy6K3KdmUdAfelUp0SeXn,0.551,0.836,10,-3.838,0,0.0524,0.00502,0.000021,0.0425,0.758,185.063,220573,4,1
995,let her go,passenger,97,2013,1KxwZYyzWNyZSRyErj2ojT,0.479,0.545,7,-7.346,1,0.0688,0.36500,0.000000,0.0963,0.243,74.897,252733,4,1
996,runnin' outta moonlight,randy houser,98,2013,6enPOmd3vSFpI4jjms8kXN,0.526,0.801,2,-3.717,1,0.0393,0.03540,0.000003,0.0637,0.769,172.024,193773,4,1
997,i'm different,2 chainz,99,2013,6J5sxraPPZ4b0CVOGAgpXj,0.936,0.702,10,-5.305,1,0.0615,0.04730,0.000000,0.0838,0.704,97.524,207040,4,1
998,still into you,paramore,100,2013,3SXXFIZel1VQQ4ENiqozxi,0.602,0.923,5,-3.763,1,0.0440,0.00980,0.000000,0.0561,0.765,136.010,216013,4,1


# get track popularity
setting the endpoint to be get track endpoint, sending a list of 50 songs in each request then addind the data to the dataframe

In [13]:
api_base_url = "https://api.spotify.com"
search_endpoint = "/v1/tracks"
i=0
j= 50
track_ids = []
tracks_popularity = []
for one in range(4):
    time.sleep(1)
    if j>190:
        track_ids = df.iloc[i:]["spotify_track_id"].tolist()
    else:
        track_ids = df.iloc[i:j]["spotify_track_id"].tolist()
    params = {
        "ids": ','.join(track_ids),
        "market": "US"
    }
      # Send the request to the API
    response = requests.get(api_base_url + search_endpoint, headers=headers, params=params)
    if response.status_code not in range(200,299):
        print(response.headers)
        print(response.headers.get('Retry-After'))
        break
    else:
        # add the data returned by the API to ids array
        data = json.loads(response.text)
        if one == 3:
            for f in range (49):
                tracks_popularity.append(data["tracks"][f]["popularity"])
        else:
            for f in range (50):
                tracks_popularity.append(data["tracks"][f]["popularity"])
    i = i+50
    j= i+50

In [14]:
updated_df = df.assign(**{"tracks_popularity": tracks_popularity})
updated_df

,song_name,artist,location_at_chart,year_of_billboard,spotify_track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,tracks_popularity
0,somebody that i used to know,gotye featuring kimbra,1,2012,4wCmqSrbyCgxEXROQE6vtV,0.864,0.495,0,-7.036,1,0.0370,0.591000,0.000133,0.0992,0.720,129.062,244973,4,72
1,call me maybe,carly rae jepsen,2,2012,3TGRqZ0a2l1LRblBkJoaDx,0.783,0.580,7,-6.548,1,0.0408,0.011400,0.000002,0.1080,0.660,120.021,193400,4,69
2,we are young,fun. featuring janelle monae,3,2012,7a86XRg84qjasly9f6bPSD,0.378,0.638,10,-5.576,1,0.0750,0.020000,0.000077,0.0849,0.735,184.086,250627,4,70
3,payphone,maroon 5 featuring wiz khalifa,4,2012,4P0osvTXoSYZZC2n8IFH3c,0.739,0.756,4,-4.828,1,0.0394,0.013600,0.000000,0.3700,0.523,110.028,231387,4,68
4,lights,ellie goulding,5,2012,4IhTXiZLKATmwhMZIb1GQN,0.440,0.040,3,-16.887,1,0.0322,0.981000,0.465000,0.1110,0.249,135.965,193749,4,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,are you gonna kiss me or not,thompson square,96,2011,2TAcwD5KcV9uY3O1tLGV15,0.485,0.701,8,-3.807,1,0.0288,0.039000,0.000000,0.0977,0.613,151.948,184920,4,62
195,animal,neon trees,97,2011,1fBl642IhJOE5U319Gy2Go,0.482,0.833,5,-5.611,1,0.0449,0.000346,0.000000,0.3650,0.740,148.039,212293,4,66
196,you and tequila,kenny chesney featuring grace potter,98,2011,1t2hJFgJyUPD2crOafMUEk,0.691,0.454,6,-9.130,1,0.0256,0.437000,0.002270,0.0783,0.350,135.839,242840,4,60
197,colder weather,zac brown band,99,2011,3U8dHeggJ8IBe0UCb1gbyB,0.667,0.381,3,-7.494,1,0.0294,0.455000,0.000000,0.1160,0.199,135.975,273893,4,67


# multipule songs ID requests at a time

In [46]:
def split_list(input_list, chunk_size):
    for i in range(0, len(input_list), chunk_size):
        yield input_list[i:i + chunk_size]
chunk_size = 100
ids_arr = df.spotify_track_id.tolist()
ids_split_list = list(split_list(ids_arr, chunk_size))

In [ ]:
# Set up the API base URL and the API endpoint for searching for songs
api_base_url = "https://api.spotify.com"
search_endpoint = "/v1/tracks"

# setting a loop for extracting spotify track id for ecah song_name in data
i=0
j= 50
#track_ids = df["spotify_track_id"].tolist()
track_ids = []

for item in df["spotify_track_id"]:
    query = id[]

    # Set up the search parameters
    params = {
        "q": query,
        "market": "US"
    }
    
    # Send the request to the API
    response = requests.get(api_base_url + search_endpoint, headers=headers, params=params)

    # Check the status code to make sure the request was successful
    if response.status_code not in range(200,299):
        print(response.headers)
        break
    else:
        # add the data returned by the API to ids array
        data = json.loads(response.text)
        track_ids.append(data["tracks"]["items"][0]["id"])
    time.sleep(3)
df["spotify_track_id"] = track_ids

# Mark these data as hits
making a new column that points if the track entered billboard (in this part they all entered) by 1 (True)

In [15]:
updated_df["entered_biilboard"] = 1
updated_df

,song_name,artist,location_at_chart,year_of_billboard,spotify_track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,tracks_popularity,entered_biilboard
0,somebody that i used to know,gotye featuring kimbra,1,2012,4wCmqSrbyCgxEXROQE6vtV,0.864,0.495,0,-7.036,1,0.0370,0.591000,0.000133,0.0992,0.720,129.062,244973,4,72,1
1,call me maybe,carly rae jepsen,2,2012,3TGRqZ0a2l1LRblBkJoaDx,0.783,0.580,7,-6.548,1,0.0408,0.011400,0.000002,0.1080,0.660,120.021,193400,4,69,1
2,we are young,fun. featuring janelle monae,3,2012,7a86XRg84qjasly9f6bPSD,0.378,0.638,10,-5.576,1,0.0750,0.020000,0.000077,0.0849,0.735,184.086,250627,4,70,1
3,payphone,maroon 5 featuring wiz khalifa,4,2012,4P0osvTXoSYZZC2n8IFH3c,0.739,0.756,4,-4.828,1,0.0394,0.013600,0.000000,0.3700,0.523,110.028,231387,4,68,1
4,lights,ellie goulding,5,2012,4IhTXiZLKATmwhMZIb1GQN,0.440,0.040,3,-16.887,1,0.0322,0.981000,0.465000,0.1110,0.249,135.965,193749,4,77,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,are you gonna kiss me or not,thompson square,96,2011,2TAcwD5KcV9uY3O1tLGV15,0.485,0.701,8,-3.807,1,0.0288,0.039000,0.000000,0.0977,0.613,151.948,184920,4,62,1
195,animal,neon trees,97,2011,1fBl642IhJOE5U319Gy2Go,0.482,0.833,5,-5.611,1,0.0449,0.000346,0.000000,0.3650,0.740,148.039,212293,4,66,1
196,you and tequila,kenny chesney featuring grace potter,98,2011,1t2hJFgJyUPD2crOafMUEk,0.691,0.454,6,-9.130,1,0.0256,0.437000,0.002270,0.0783,0.350,135.839,242840,4,60,1
197,colder weather,zac brown band,99,2011,3U8dHeggJ8IBe0UCb1gbyB,0.667,0.381,3,-7.494,1,0.0294,0.455000,0.000000,0.1160,0.199,135.975,273893,4,67,1


In [16]:
# updated_df.to_csv("../Data/03 billboard_scraped_w featuers_popular.csv")
# updated_df.to_csv("../Data/05 scraped_data_added_w_featuers_popular.csv")